In [1]:
import sys
import os
import random
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src')) 

from load_data import DataLoader
import path 

from Hypothesis_Analysis import hypothesis
from Data_preparation import DataPreparation

In [2]:
# Loading and reading text file using pandas
csv_path = path.get_clead_data()
data_load = DataLoader(csv_path)
df = data_load.load_csv_data()

In [3]:
df.head()

,Unnamed: 0,TransactionMonth,IsVATRegistered,Citizenship,MaritalStatus,Gender,Province,PostalCode,VehicleType,RegistrationYear,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,2015-03-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,2015-05-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,2015-07-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,2015-05-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,4,2015-07-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


## A/B Hypothesis Testing
For this analysis, "risk" will be quantified by two metrics: Claim Frequency (proportion of policies with at least one claim) and Claim Severity (the average amount of a claim, given a claim occurred). "Margin" is defined as (TotalPremium - TotalClaims).


In [ ]:
# Performs ANOVA test across groups. Filters out any groups with only one unique value.
 #       Returns p-value or warning message.

In [5]:
# Convert TotalPremium and TotalClaims to numeric
df["TotalPremium"] = pd.to_numeric(df["TotalPremium"], errors="coerce")
df["TotalClaims"] = pd.to_numeric(df["TotalClaims"], errors="coerce")

In [6]:
# Create new columns
df["HasClaim"] = df["TotalClaims"] > 0
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]

### 1. ANOVA: Risk differences across Provinces (Claim Frequency)

In [7]:
from scipy.stats import ttest_ind, f_oneway
province_groups = [group["HasClaim"].astype(int) for _, group in df.groupby("Province")]
anova_province_risk = f_oneway(*province_groups)

In [8]:
anova_province_risk.pvalue

np.float64(5.91210036318191e-19)

In [9]:
# 2. ANOVA: Risk differences between Zip Codes (Claim Frequency)
zip_groups = [group["HasClaim"].astype(int) for _, group in df.groupby("PostalCode")]
anova_zip_risk = f_oneway(*zip_groups)

In [10]:
# 3. ANOVA: Margin differences between Zip Codes
margin_groups = [group["Margin"].dropna() for _, group in df.groupby("PostalCode")]
anova_zip_margin = f_oneway(*margin_groups)

### Performs T-test between Male and Female on claim risk

In [9]:
# 4. T-test: Risk differences by Gender
men = df[df["Gender"] == "Male"]["HasClaim"].astype(int)
women = df[df["Gender"] == "Female"]["HasClaim"].astype(int)
ttest_gender_risk = ttest_ind(men, women, nan_policy="omit")

In [10]:
# Collect results
results = {
    "Province Risk ANOVA p-value": anova_province_risk.pvalue,
    "Zip Risk ANOVA p-value": anova_zip_risk.pvalue,
    "Zip Margin ANOVA p-value": anova_zip_margin.pvalue,
    "Gender Risk T-test p-value": ttest_gender_risk.pvalue
}

results

{'Province Risk ANOVA p-value': np.float64(5.91210036318191e-19),
 'Zip Risk ANOVA p-value': np.float64(2.9076595484940585e-30),
 'Zip Margin ANOVA p-value': np.float64(0.9976859758015036),
 'Gender Risk T-test p-value': np.float64(0.8404980845002314)}

### To perform controlled A/B testing on insurance plan features (e.g., AlarmImmobiliser, TrackingDevice, CoverType, etc.), we need to follow a clear and reproducible approach.

In [11]:
hypoth = hypothesis(df)

In [12]:
hypoth.compute_kpis()

,Unnamed: 0,TransactionMonth,IsVATRegistered,Citizenship,MaritalStatus,Gender,Province,PostalCode,VehicleType,RegistrationYear,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,HasClaim,Margin,ClaimSeverity
0,0,2015-03-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0,21.929825,0.0
1,1,2015-05-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0,21.929825,0.0
2,2,2015-07-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0,0.000000,0.0
3,3,2015-05-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0,512.848070,0.0
4,4,2015-07-01 00:00:00,True,,Not specified,Not specified,Gauteng,1459,Passenger Vehicle,2004,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,1000093,2015-04-01 00:00:00,False,ZW,Single,Male,Western Cape,7493,Passenger Vehicle,2013,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,0,347.235175,0.0
1000094,1000094,2015-06-01 00:00:00,False,ZW,Single,Male,Western Cape,7493,Passenger Vehicle,2013,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,0,347.235175,0.0
1000095,1000095,2015-08-01 00:00:00,False,ZW,Single,Male,Western Cape,7493,Passenger Vehicle,2013,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,0,347.235175,0.0
1000096,1000096,2014-07-01 00:00:00,False,ZW,Single,Male,Western Cape,7493,Passenger Vehicle,2013,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0,0,2.315000,0.0


In [13]:
group_a, group_b = hypoth.create_ab_groups("Gender", "Male", "Female")

In [14]:
p_value = hypoth.compare_kpis(group_a, group_b, "Margin")
print("P-value for Margin difference between Male and Female:", p_value)

P-value for Margin difference between Male and Female: 0.8015464193501282


In [15]:
results, size_a, size_b = hypoth.run_ab_test("Gender", "Male", "Female")

In [16]:
print(f"Group Sizes -> A: {size_a}, B: {size_b}")
print("P-Values for KPIs:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

Group Sizes -> A: 42817, B: 6755
P-Values for KPIs:
HasClaim (p-value): 0.8372
ClaimSeverity (p-value): 0.7670
Margin (p-value): 0.8015


###	Statistical Testing 

In [ ]:
hypoth.perform_t_test()

In [ ]:
hypoth.perform_chi_squared()

In [20]:
# Example 1: T-Test between Male and Female on all KPIs
results_gender = hypoth.run_all_tests(feature="Gender", group_a_val="Male", group_b_val="Female")
print(results_gender)

     Test Feature Group A Group B            KPI   P-Value Significant
0  T-Test  Gender    Male  Female       HasClaim  0.837187          No
1  T-Test  Gender    Male  Female  ClaimSeverity  0.766966          No
2  T-Test  Gender    Male  Female         Margin  0.801546          No


In [21]:
# 2: Chi-Squared test for Province effect on claim frequency
results_province = hypoth.run_all_tests(feature="Province")
print(results_province)

          Test   Feature       KPI       P-Value Significant
0  Chi-Squared  Province  HasClaim  5.925511e-19         Yes


In [22]:
# 2: Chi-Squared test for Province effect on claim frequency
results_PostalCode = hypoth.run_all_tests(feature="PostalCode")
print(results_PostalCode)

          Test     Feature       KPI       P-Value Significant
0  Chi-Squared  PostalCode  HasClaim  3.152172e-30         Yes


# Statistical Modeling
### Data Preparation

In [11]:
df.isna().sum()

Unnamed: 0                  0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
MaritalStatus               0
Gender                      0
Province                    0
PostalCode                  0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
SumInsured                  0
TermFrequency               0
CalculatedPremiumPerTerm    0
ExcessSelected              0
CoverCategory               0
CoverType                   0
CoverGroup                  0
Section                     0
Product                     0
StatutoryClass              0
StatutoryRiskType           0
TotalPremium                0
TotalClaim

In [24]:
# Convert the column to datetime format
df["TransactionMonth"] = pd.to_datetime(df["TransactionMonth"])

In [ ]:
df = df.drop("Unnamed: 0", axis=1)

In [43]:
df1 = df.drop(columns=['Citizenship', 'MaritalStatus', 'Gender', 'make','CoverType', 'CoverGroup', 'Section', 'Model', 'TermFrequency',
                       'StatutoryClass', 'CapitalOutstanding', 'ExcessSelected', 'CoverCategory', 'StatutoryRiskType', 
                       'VehicleIntroDate', 'bodytype' ], axis=1)

In [44]:
# Automatically detect categorical (object or category) columns
categorical_cols = df1.select_dtypes(include=["object", "category"]).columns

# Count values for each categorical column
for col in categorical_cols:
    print(f"\nValue counts for '{col}':")
    print(df1[col].value_counts())


Value counts for 'Province':
Province
Gauteng          393865
Western Cape     170796
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52718
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64

Value counts for 'VehicleType':
VehicleType
Passenger Vehicle    934150
Medium Commercial     53985
Heavy Commercial       7401
Light Commercial       3897
Bus                     665
Name: count, dtype: int64

Value counts for 'AlarmImmobiliser':
AlarmImmobiliser
Yes    999861
No        237
Name: count, dtype: int64

Value counts for 'TrackingDevice':
TrackingDevice
No     656617
Yes    343481
Name: count, dtype: int64

Value counts for 'Product':
Product
Mobility Commercial Cover: Monthly    915028
Mobility Metered Taxis: Monthly        79272
Bridge Taxi Finance: Monthly            5254
Standalone Passenger Liability           544
Name: count, dtype: int64


In [45]:

preparation = DataPreparation(df1)

In [46]:
# Imputes missing numerical and categorical values.
preparation.handle_missing_data()

,TransactionMonth,IsVATRegistered,Province,PostalCode,VehicleType,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,AlarmImmobiliser,TrackingDevice,SumInsured,CalculatedPremiumPerTerm,Product,TotalPremium,TotalClaims,Margin,ClaimFlag
0,2015-03-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,21.929825,0.0,21.929825,0.0
1,2015-05-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,21.929825,0.0,21.929825,0.0
2,2015-07-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,0.000000,0.0,0.000000,0.0
3,2015-05-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,119300.00,584.6468,Mobility Metered Taxis: Monthly,512.848070,0.0,512.848070,0.0
4,2015-07-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,119300.00,584.6468,Mobility Metered Taxis: Monthly,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,2015-04-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0.0
1000094,2015-06-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0.0
1000095,2015-08-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0.0
1000096,2014-07-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,5000000.00,2.6391,Mobility Commercial Cover: Monthly,2.315000,0.0,2.315000,0.0


In [47]:
# Create new features relevant to TotalPremium and TotalClaims
preparation.feature_engineering()

,TransactionMonth,IsVATRegistered,Province,PostalCode,VehicleType,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,AlarmImmobiliser,TrackingDevice,SumInsured,CalculatedPremiumPerTerm,Product,TotalPremium,TotalClaims,Margin,ClaimFlag
0,2015-03-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,21.929825,0.0,21.929825,0
1,2015-05-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,21.929825,0.0,21.929825,0
2,2015-07-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,0.01,25.0000,Mobility Metered Taxis: Monthly,0.000000,0.0,0.000000,0
3,2015-05-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,119300.00,584.6468,Mobility Metered Taxis: Monthly,512.848070,0.0,512.848070,0
4,2015-07-01,True,Gauteng,1459.0,Passenger Vehicle,2004.0,6.0,2597.0,130.0,4.0,Yes,No,119300.00,584.6468,Mobility Metered Taxis: Monthly,0.000000,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,2015-04-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0
1000094,2015-06-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0
1000095,2015-08-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,500000.00,395.8481,Mobility Commercial Cover: Monthly,347.235175,0.0,347.235175,0
1000096,2014-07-01,False,Western Cape,7493.0,Passenger Vehicle,2013.0,4.0,2693.0,110.0,4.0,Yes,No,5000000.00,2.6391,Mobility Commercial Cover: Monthly,2.315000,0.0,2.315000,0


In [48]:
# One-hot encodes categorical variables.
preparation.encode_categorical()

,TransactionMonth,IsVATRegistered,PostalCode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,SumInsured,CalculatedPremiumPerTerm,...,Province_Western Cape,VehicleType_Heavy Commercial,VehicleType_Light Commercial,VehicleType_Medium Commercial,VehicleType_Passenger Vehicle,AlarmImmobiliser_Yes,TrackingDevice_Yes,Product_Mobility Commercial Cover: Monthly,Product_Mobility Metered Taxis: Monthly,Product_Standalone Passenger Liability
0,2015-03-01,True,1459.0,2004.0,6.0,2597.0,130.0,4.0,0.01,25.0000,...,False,False,False,False,True,True,False,False,True,False
1,2015-05-01,True,1459.0,2004.0,6.0,2597.0,130.0,4.0,0.01,25.0000,...,False,False,False,False,True,True,False,False,True,False
2,2015-07-01,True,1459.0,2004.0,6.0,2597.0,130.0,4.0,0.01,25.0000,...,False,False,False,False,True,True,False,False,True,False
3,2015-05-01,True,1459.0,2004.0,6.0,2597.0,130.0,4.0,119300.00,584.6468,...,False,False,False,False,True,True,False,False,True,False
4,2015-07-01,True,1459.0,2004.0,6.0,2597.0,130.0,4.0,119300.00,584.6468,...,False,False,False,False,True,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,2015-04-01,False,7493.0,2013.0,4.0,2693.0,110.0,4.0,500000.00,395.8481,...,True,False,False,False,True,True,False,True,False,False
1000094,2015-06-01,False,7493.0,2013.0,4.0,2693.0,110.0,4.0,500000.00,395.8481,...,True,False,False,False,True,True,False,True,False,False
1000095,2015-08-01,False,7493.0,2013.0,4.0,2693.0,110.0,4.0,500000.00,395.8481,...,True,False,False,False,True,True,False,True,False,False
1000096,2014-07-01,False,7493.0,2013.0,4.0,2693.0,110.0,4.0,5000000.00,2.6391,...,True,False,False,False,True,True,False,True,False,False


In [51]:
df1.columns

Index(['TransactionMonth', 'IsVATRegistered', 'Province', 'PostalCode',
       'VehicleType', 'RegistrationYear', 'Cylinders', 'cubiccapacity',
       'kilowatts', 'NumberOfDoors', 'AlarmImmobiliser', 'TrackingDevice',
       'SumInsured', 'CalculatedPremiumPerTerm', 'Product', 'TotalPremium',
       'TotalClaims', 'Margin', 'ClaimFlag'],
      dtype='object')

In [ ]:
## Splits the dataset into training and testing sets
preparation.split_data('TotalClaims')

[       TransactionMonth  IsVATRegistered  PostalCode  RegistrationYear  \
 448476       2015-04-01            False      7441.0            2008.0   
 629423       2015-04-01            False      1401.0            2010.0   
 293724       2015-05-01            False       122.0            2014.0   
 793115       2015-06-01            False       477.0            2006.0   
 120459       2015-07-01            False      2000.0            2010.0   
 ...                 ...              ...         ...               ...   
 259178       2015-02-01            False      1724.0            2014.0   
 365838       2015-04-01            False       400.0            2009.0   
 131932       2014-12-01            False      1559.0            2007.0   
 671155       2015-08-01            False      7785.0            2014.0   
 121958       2015-03-01            False      1724.0            2012.0   
 
         Cylinders  cubiccapacity  kilowatts  NumberOfDoors  SumInsured  \
 448476        4.0     